<a href="https://colab.research.google.com/github/AnshulAgrvl/Tesla_PMM/blob/main/AITP_Gradio_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio supabase pandas textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 8.5 MB/s eta 0:00:00


To securely use your Supabase credentials, you'll need to store your `SUPABASE_URL` and `SUPABASE_KEY` as secrets in Google Colab. You can do this by clicking on the '🔑' icon in the left panel and adding them with the exact names `SUPABASE_URL` and `SUPABASE_KEY`.

In [2]:
from supabase import create_client
from google.colab import userdata

# Retrieve Supabase credentials from Colab secrets
SUPABASE_URL = userdata.get('SUPABASE_URL')
SUPABASE_KEY = userdata.get('SUPABASE_KEY')

# Initialize the Supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

print("Supabase client initialized successfully!")

Supabase client initialized successfully!


In [3]:
import pandas as pd

def fetch_data():
    """Fetches data from the 'tesla_comments' table and returns it as a Pandas DataFrame."""
    try:
        # Query the 'tesla_comments' table
        response = supabase.table('tesla_comments').select('comment_id, video_id, parent_comment_id, comment_type, author, text, like_count, published_at, updated_at, brand, label').execute()

        # Check for errors in the Supabase response
        if response.data:
            df = pd.DataFrame(response.data)
            return df
        else:
            print("No data found or an error occurred:", response.error)
            return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

print("fetch_data() function defined.")

fetch_data() function defined.


In [4]:
df = fetch_data()
if not df.empty:
    print("First 5 rows of the DataFrame:")
    print(df.head())
else:
    print("DataFrame is empty. No data to display.")

First 5 rows of the DataFrame:
                                          comment_id     video_id  \
0                         UgyP-XFboanu53VIKe54AaABAg  rJgffrenFCU   
1  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMsC3_CL-g8  rJgffrenFCU   
2  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMsXCLkHaSQ  rJgffrenFCU   
3  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMsmvlXQOZT  rJgffrenFCU   
4  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMssOYm-hAt  rJgffrenFCU   

            parent_comment_id comment_type                    author  \
0                        None          top         @vegasteslacarmen   
1  UgyP-XFboanu53VIKe54AaABAg        reply        @stevepriority4219   
2  UgyP-XFboanu53VIKe54AaABAg        reply                 @SRTNicky   
3  UgyP-XFboanu53VIKe54AaABAg        reply  @NotTheOneToBeMessedWith   
4  UgyP-XFboanu53VIKe54AaABAg        reply         @vegasteslacarmen   

                                                text  like_count  \
0  WTF 🤬 sorry guys - a lot of my B roll videos e... 

In [5]:
from textblob import TextBlob

def calculate_sentiment_scores(df):
    """Calculates sentiment scores (polarity) for comments using TextBlob and adds them as a new column."""
    if 'text' not in df.columns:
        print("Error: DataFrame must contain a 'text' column.")
        return df

    # Calculate sentiment polarity for each comment
    df['sentiment_score'] = df['text'].apply(lambda comment: TextBlob(str(comment)).sentiment.polarity)
    return df

# Apply the function to the existing DataFrame
if not df.empty:
    df_with_sentiment = calculate_sentiment_scores(df.copy())
    print("DataFrame with sentiment scores (first 5 rows):")
    print(df_with_sentiment.head())
else:
    print("DataFrame is empty, cannot calculate sentiment scores.")

DataFrame with sentiment scores (first 5 rows):
                                          comment_id     video_id  \
0                         UgyP-XFboanu53VIKe54AaABAg  rJgffrenFCU   
1  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMsC3_CL-g8  rJgffrenFCU   
2  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMsXCLkHaSQ  rJgffrenFCU   
3  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMsmvlXQOZT  rJgffrenFCU   
4  UgyP-XFboanu53VIKe54AaABAg.AMs-fmWkkDjAMssOYm-hAt  rJgffrenFCU   

            parent_comment_id comment_type                    author  \
0                        None          top         @vegasteslacarmen   
1  UgyP-XFboanu53VIKe54AaABAg        reply        @stevepriority4219   
2  UgyP-XFboanu53VIKe54AaABAg        reply                 @SRTNicky   
3  UgyP-XFboanu53VIKe54AaABAg        reply  @NotTheOneToBeMessedWith   
4  UgyP-XFboanu53VIKe54AaABAg        reply         @vegasteslacarmen   

                                                text  like_count  \
0  WTF 🤬 sorry guys - a lot of my B 

In [7]:
df_with_sentiment['published_at'] = pd.to_datetime(df_with_sentiment['published_at'])
# Round down to the nearest hour
df_with_sentiment['published_at_hourly'] = df_with_sentiment['published_at'].dt.floor('h')

print("DataFrame with 'published_at' converted to datetime and rounded to the nearest hour:")
display(df_with_sentiment[['published_at', 'published_at_hourly']].head())

DataFrame with 'published_at' converted to datetime and rounded to the nearest hour:


,published_at,published_at_hourly
0,2025-09-09 22:33:11+00:00,2025-09-09 22:00:00+00:00
1,2025-09-10 00:21:26+00:00,2025-09-10 00:00:00+00:00
2,2025-09-10 03:26:07+00:00,2025-09-10 03:00:00+00:00
3,2025-09-10 05:52:16+00:00,2025-09-10 05:00:00+00:00
4,2025-09-10 06:40:02+00:00,2025-09-10 06:00:00+00:00


In [8]:
# Sort by sentiment_score in descending order to get the highest sentiment comments
df_top_pos = df_with_sentiment.sort_values(by='sentiment_score', ascending=False).head(2)

print("Top 2 comments with the highest sentiment scores:")
display(df_top_pos)

Top 2 comments with the highest sentiment scores:


,comment_id,video_id,parent_comment_id,comment_type,author,text,like_count,published_at,updated_at,brand,label,sentiment_score,published_at_hourly
407,UgwXbO86RyUp7QKUUxd4AaABAg,6ltU9q1pKKM,None,top,@Bob-v3d8t,"Elon is the best CEO on the planet, this hit p...",1,2026-01-10 01:53:05+00:00,2026-01-10T01:53:05+00:00,Tesla,critical,1.0,2026-01-10 01:00:00+00:00
334,UgzI09VbRz1hDNszbSx4AaABAg,PBC4kkjS3P0,None,top,@liquidskateboard,"Damn, what a privilege to be mentioned here. Y...",257,2025-10-13 09:32:21+00:00,2025-10-13T09:32:21+00:00,Tesla,critical,1.0,2025-10-13 09:00:00+00:00


In [9]:
# Sort by sentiment_score in ascending order to get the lowest sentiment comments
df_top_neg = df_with_sentiment.sort_values(by='sentiment_score', ascending=True).head(2)

print("Top 2 comments with the lowest sentiment scores:")
display(df_top_neg)

Top 2 comments with the lowest sentiment scores:


,comment_id,video_id,parent_comment_id,comment_type,author,text,like_count,published_at,updated_at,brand,label,sentiment_score,published_at_hourly
310,Ugx0MzUpi1movYVB5ix4AaABAg,PBC4kkjS3P0,None,top,@axelotl86,They are especially a bad deal because Musk is...,3,2025-10-14 05:52:08+00:00,2025-10-14T05:52:08+00:00,Tesla,critical,-1.0000,2025-10-14 05:00:00+00:00
129,UgynMxAO-PXfes2YjK14AaABAg,b6B7muYkuXM,None,top,@extramedium3920,.07 a kWh… crazy!! try 43 in Ca. bracing for it,0,2026-01-08 02:31:35+00:00,2026-01-08T02:31:35+00:00,Tesla,fanboy,-0.9375,2026-01-08 02:00:00+00:00


In [13]:
# Convert 'published_at' to datetime and create a daily timestamp
df_with_sentiment['published_at_daily'] = df_with_sentiment['published_at'].dt.floor('D')

# Aggregate by day to calculate the average sentiment_score
daily_sentiment = df_with_sentiment.groupby('published_at_daily')['sentiment_score'].mean().reset_index()

# Ensure the daily sentiment data is sorted by date for correct rolling average calculation
daily_sentiment = daily_sentiment.sort_values(by='published_at_daily')

# Calculate a 7-day Rolling Average and save it as 'trend_score'
daily_sentiment['trend_score'] = daily_sentiment['sentiment_score'].rolling(window=7, min_periods=1).mean()

print("Daily sentiment with 7-day rolling average (first 5 rows):")
display(daily_sentiment.head())

Daily sentiment with 7-day rolling average (first 5 rows):


,published_at_daily,sentiment_score,trend_score
0,2025-03-07 00:00:00+00:00,0.062500,0.062500
1,2025-09-09 00:00:00+00:00,-0.266667,-0.102083
2,2025-09-10 00:00:00+00:00,0.210758,0.002197
3,2025-09-14 00:00:00+00:00,0.000000,0.001648
4,2025-09-15 00:00:00+00:00,0.084068,0.018132


In [19]:
import gradio as gr

def get_sentiment_data():
    # Ensuring clean data for plotting
    cleaned_daily_sentiment = daily_sentiment.dropna(subset=['published_at_daily', 'sentiment_score', 'trend_score'])
    print("\n--- Data passed to Gradio LinePlot (head): ---")
    print(cleaned_daily_sentiment.head())
    print("--------------------------------------------------")
    return cleaned_daily_sentiment

with gr.Blocks() as demo:
    gr.Markdown("# Vibe Check (Monitoring)")
    gr.LinePlot(
        value=get_sentiment_data,
        x="published_at_daily",
        y="sentiment_score",
        title="Daily Sentiment Over Time",
        height=400,
        width=800,
        x_title="Published At (Daily)",
        y_title="Sentiment Score"
    )
    gr.LinePlot(
        value=get_sentiment_data,
        x="published_at_daily",
        y="trend_score",
        title="7-Day Rolling Average Sentiment Over Time",
        height=400,
        width=800,
        x_title="Published At (Daily)",
        y_title="Trend Score"
    )
    gr.Markdown("## The Drill-Down")
    with gr.Row():
        gr.DataFrame(value=df_top_pos[['text']], label='😇 High Vibes (Top Positive)', row_count=(2, 'fixed'), col_count=(1, 'fixed'))
        gr.DataFrame(value=df_top_neg[['text']], label='💀 Toxic Vibes (Top Negative)', row_count=(2, 'fixed'), col_count=(1, 'fixed'))

demo.launch()


--- Data passed to Gradio LinePlot (head): ---
         published_at_daily  sentiment_score  trend_score
0 2025-03-07 00:00:00+00:00         0.062500     0.062500
1 2025-09-09 00:00:00+00:00        -0.266667    -0.102083
2 2025-09-10 00:00:00+00:00         0.210758     0.002197
3 2025-09-14 00:00:00+00:00         0.000000     0.001648
4 2025-09-15 00:00:00+00:00         0.084068     0.018132
--------------------------------------------------

--- Data passed to Gradio LinePlot (head): ---
         published_at_daily  sentiment_score  trend_score
0 2025-03-07 00:00:00+00:00         0.062500     0.062500
1 2025-09-09 00:00:00+00:00        -0.266667    -0.102083
2 2025-09-10 00:00:00+00:00         0.210758     0.002197
3 2025-09-14 00:00:00+00:00         0.000000     0.001648
4 2025-09-15 00:00:00+00:00         0.084068     0.018132
--------------------------------------------------
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatica